In [13]:
"""
           ======================================================
           =                DATA CLEANING                       =
           ======================================================
"""
import pandas as pd
from prophet import Prophet
# Importing the dataset 
df = pd.read_csv('/workspaces/FUTURE_ML_01/data/mock_kaggle.csv')

# the data column change to English for understanding these are the original portuguese headers of the data set
# which the translations makes analysis easier
df = df.rename(columns={
    'data':'Date',
    'venda':'Sales',
    'estoque':'Stock',
    'preco':'Price'
})

# making the date proper date time objects allows python to understand months, years and seasons 
df['Date'] = pd.to_datetime(df['Date'])

# Sort the data by Date
# Time-series models require data to be in chronological order
df = df.sort_values('Date').reset_index(drop=True)

# Daily Aggregation. It is used to transform "transactional data" (multiple rows per day) into "time-series data" (one single row per day).
# Machine Learning models for forecasting (like Prophet or ARIMA) require a consistent timeline. 
# They need to see exactly one data point per day to understand the trend correctly.
# If there are multiple entries for the same day, we sum them up
df_cleaned = df.groupby('Date').agg({
    'Sales': 'sum',
    'Stock': 'last', # Take the stock level at the end of the day
    'Price': 'mean'  # Take the average price for that day
}).reset_index()

# 6. Save the cleaned data to a new file
df_cleaned.to_csv('cleaned_retail_data.csv', index=False)
print("Data cleaning complete!")
print(f"Dataset range: {df_cleaned['Date'].min()} to {df_cleaned['Date'].max()}")
print(df_cleaned.head())


Data cleaning complete!
Dataset range: 2014-01-01 00:00:00 to 2016-07-31 00:00:00
        Date  Sales  Stock  Price
0 2014-01-01      0   4972   1.29
1 2014-01-02     70   4902   1.29
2 2014-01-03     59   4843   1.29
3 2014-01-04     93   4750   1.29
4 2014-01-05     96   4654   1.29


In [ ]:
"""
                 ======================================================
                 =          EDA & VISUALIZATION                       =
                 ======================================================
"""
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Load the cleaned data
df = pd.read_csv('cleaned_retail_data.csv')
df['Date'] = pd.to_datetime(df['Date'])

# 1. Create Time-Based columns for analysis
df['Month'] = df['Date'].dt.month_name()
df['DayName'] = df['Date'].dt.day_name()

# Set the visual style
plt.style.use('fivethirtyeight')
fig, axes = plt.subplots(2, 2, figsize=(20, 12))

# --- PLOT 1: Daily Sales Trend ---
sns.lineplot(ax=axes[0, 0], data=df, x='Date', y='Sales', color='#30a2da')
axes[0, 0].set_title('Daily Sales Volume (2014-2016)', fontsize=16)

# --- PLOT 2: Sales by Day of the Week ---
day_order = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
sns.barplot(ax=axes[0, 1], data=df, x='DayName', y='Sales', order=day_order, hue='DayName', palette='viridis', legend=False)
axes[0, 1].set_title('Average Sales per Weekday', fontsize=16)

# --- PLOT 3: Price vs. Sales (Correlation) ---
sns.scatterplot(ax=axes[1, 0], data=df, x='Price', y='Sales', alpha=0.5, color='#fc4f30')
axes[1, 0].set_title('Price Sensitivity (Sales vs. Price)', fontsize=16)

# --- PLOT 4: Stock Levels Over Time ---
sns.lineplot(ax=axes[1, 1], data=df, x='Date', y='Stock', color='#e5ae38')
axes[1, 1].set_title('Inventory Levels (Stock on Hand)', fontsize=16)

plt.tight_layout()
plt.show()

# 5. Print a quick Correlation Matrix
print("--- Statistical Correlation ---")
print(df[['Sales', 'Price', 'Stock']].corr())